In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import pandas as pd
import re
import fnmatch
import shutil
from os.path import join
%matplotlib inline
plt.rcParams["figure.figsize"] = (25, 25)

In [2]:
#path = '/mnt/data/data/ub-im/Sort-result'
#img = pd.DataFrame(columns=('id', 'kclass', 'code'))
#for x in range(0,6):
#    files = os.listdir(path + '/' + str(x))
#    for file in files:
#        labels = file.replace('.jpg', '').split('_')
#        if img[img.id == int(labels[2])].empty:
#            code = '----'
#            code = code[0:int(labels[3])]   + labels[4] + code[int(labels[3])+1: len(code)] 
#            img = img.append([{'id':int(labels[2]), 'kclass': int(labels[0]), 'code': code}], ignore_index=True)
#        else:
#            index = img[img.id == int(labels[2])].index
#            code =  str(img.iloc[index, 2].values[0])
#            img.iloc[index, 2] = code[0:int(labels[3])]   + labels[4] + code[int(labels[3])+1: len(code)]
            #img[img.id == int(labels[2])]['code'] = code[0:int(labels[4])]   + labels[4] + code[int(labels[4]) + 1: len(code)]

    

In [116]:
#df = img[img.kclass == 2]
#df.describe()
#df.groupby(['code']).size().reset_index(name='counts').sort_values(by='counts', ascending = False)

,id,kclass,code
count,1329,1329,1329
unique,1329,1,107
top,2045,2,3232
freq,1,1329,684


In [5]:
#смещение изображения
def img_offset(segment):
    if segment == 0 or segment == 3:
        return segment
    elif segment == 1:
        return 2
    elif segment == 2:
        return 1
    else:
        return 0

#конвертация изображения
def img_convert(path):
    im_b = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (224, 224))
    mn = np.mean(im_b)
    if mn > 124:
        k = (mn-124)*0.75/124 +0.75
        im_b = cv2.multiply(im_b, k)
    im_b = cv2.equalizeHist(im_b)
    im = cv2.GaussianBlur(im_b, (1, 1), 0)
    kernel = np.array([[0, -1, 0],
                       [-1, 4.5, -1],
                       [0, -1, 0]])
    im_f = cv2.filter2D(im, -3, kernel, (-5,-5))
    im_ad = cv2.adaptiveThreshold(im_f, 90, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 71, 3)
    
    struct_element = cv2.getStructuringElement(cv2.MORPH_RECT, (1,1))
    im_erod = cv2.erode(im_ad, struct_element)
    im_dilate = cv2.dilate(im_erod, struct_element)
    ot_tr, im_dilate = cv2.threshold(im_dilate, 0, 120, cv2.THRESH_BINARY|cv2.THRESH_OTSU)
    hi_tr = ot_tr
    lo_tr = ot_tr * 0.8
    im_can = cv2.Canny(im_dilate, lo_tr, hi_tr)
    im_can = cv2.GaussianBlur(im_can, (5, 5), 0)
    return im_can

def img_write(im, path, fname, im_id, offset, pclass, cclass):
    code = '----'
    if fname != '':
        keys = fname.split('_')
        code = keys[3].replace('.jpg', '')
    code = code[0:offset] + cclass + code[offset + 1: len(code)]    
    fname = pclass + '_IMG-K2-UB_' + im_id + '_' + code + '.jpg'
    print(path + fname)
    cv2.imwrite(path + fname, im, [int(cv2.IMWRITE_JPEG_QUALITY), 100])

In [6]:
img_path = '/mnt/data/data/ub-im/Imgtrain/'
path = '/mnt/data/data/ub-im/Sort-result'
flag = False
for x in range(0,6):
    files = os.listdir(path + '/' + str(x))  
    for file in files:
        flag = False
        labels = file.replace('.jpg', '').split('_')
        print(file)
        img_files = os.listdir(img_path)
        fpath = path + '/' + str(x) + '/' + file
        for f in img_files:
            if fnmatch.fnmatch(f, '*_' + labels[2] + '_*'):
                im = cv2.imread(img_path + f,  cv2.IMREAD_GRAYSCALE)
                offset = img_offset(int(labels[3]))
                if offset == 0:
                    im[0:224, 0:224] = img_convert(fpath)
                else:
                    im[0:224, offset*226:offset*226 + 224] = img_convert(fpath)
                img_write(im, img_path, f, labels[2], offset, labels[0], labels[4])
                os.remove(img_path + f)
                flag = True
                break
        if not flag:
            im =np.zeros((224, 224*4 + 6), np.uint8)
            offset = img_offset(int(labels[3]))
            if offset == 0:
                im[0:224, 0:224] = img_convert(fpath)
            else:
                im[0:224, offset*226:offset*226 + 224] = img_convert(fpath)
            img_write(im, img_path, '', labels[2], offset, labels[0], labels[4])  

2_IM-K2-UB_738697636_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_738697636_--0-.jpg
4_IM-K2-UB_2145141285_3_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_2145141285_---0.jpg
4_IM-K2-UB_517604657_3_1.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_517604657_---1.jpg
2_IM-K2-UB_5086_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_5086_--0-.jpg
4_IM-K2-UB_951658279_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_951658279_--0-.jpg
4_IM-K2-UB_797059895_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_797059895_--0-.jpg
2_IM-K2-UB_1848591454_3_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_1848591454_---0.jpg
4_IM-K2-UB_3869326989_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_3869326989_--0-.jpg
4_IM-K2-UB_924450034_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_924450034_--0-.jpg
2_IM-K2-UB_896385378_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_896385378_--0-.jpg
4_IM-K2-UB_675218392_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_675218392_--0-.jpg
4_IM-K2-UB_574

4_IM-K2-UB_2533395402_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_2533395402_--0-.jpg
2_IM-K2-UB_1600_0_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_1600_00--.jpg
2_IM-K2-UB_19135_3_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_19135_--00.jpg
4_IM-K2-UB_5116_3_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_5116_---0.jpg
4_IM-K2-UB_2283014954_3_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_2283014954_---0.jpg
4_IM-K2-UB_2540883668_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_2540883668_--0-.jpg
4_IM-K2-UB_1592019772_3_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1592019772_---0.jpg
2_IM-K2-UB_12355_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_12355_--0-.jpg
2_IM-K2-UB_9574_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_9574_--0-.jpg
4_IM-K2-UB_426285994_3_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_426285994_--00.jpg
2_IM-K2-UB_19694_3_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_19694_--00.jpg
4_IM-K2-UB_1402760463_3_0.jpg
/mnt/data/data/ub-im/Imgtr

/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_14509_--0-.jpg
2_IM-K2-UB_16447_3_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_16447_0--0.jpg
4_IM-K2-UB_892551964_3_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_892551964_--00.jpg
4_IM-K2-UB_4195225624_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_4195225624_--00.jpg
4_IM-K2-UB_453661135_3_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_453661135_--00.jpg
2_IM-K2-UB_119_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_119_--0-.jpg
2_IM-K2-UB_8531_3_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_8531_---0.jpg
2_IM-K2-UB_8166_3_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_8166_00-0.jpg
2_IM-K2-UB_2283_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_2283_--0-.jpg
4_IM-K2-UB_3920456684_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_3920456684_--00.jpg
2_IM-K2-UB_13517_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_13517_--0-.jpg
2_IM-K2-UB_3647629594_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_3647629594_-00-.jpg
4_IM-

4_IM-K2-UB_1592019772_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1592019772_--00.jpg
4_IM-K2-UB_2336575343_3_1.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_2336575343_---1.jpg
4_IM-K2-UB_1557921693_3_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1557921693_---0.jpg
4_IM-K2-UB_4109390990_3_1.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_4109390990_---1.jpg
2_IM-K2-UB_18491_0_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_18491_0000.jpg
4_IM-K2-UB_9505_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_9505_--0-.jpg
2_IM-K2-UB_17501_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_17501_--0-.jpg
4_IM-K2-UB_9505_3_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_9505_--00.jpg
2_IM-K2-UB_11559_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_11559_--0-.jpg
4_IM-K2-UB_2746290909_1_0.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_2746290909_--0-.jpg
4_IM-K2-UB_1166164613_1_1.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1166164613_--1-.jpg
2_IM-K2-UB_7396_1_1.jpg
/mnt/data/data/ub-

/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1848992832_--1-.jpg
2_IM-K2-UB_18112_1_1.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_18112_--1-.jpg
2_IM-K2-UB_13213_3_1.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_13213_---1.jpg
2_IM-K2-UB_11391_1_1.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_11391_--1-.jpg
4_IM-K2-UB_4211668223_1_1.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_4211668223_--1-.jpg
4_IM-K2-UB_2397632283_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_2397632283_5--0.jpg
4_IM-K2-UB_597946072_1_1.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_597946072_--1-.jpg
4_IM-K2-UB_3165273617_3_1.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_3165273617_--11.jpg
4_IM-K2-UB_1369552749_3_1.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1369552749_---1.jpg
4_IM-K2-UB_994132688_1_1.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_994132688_--1-.jpg
2_IM-K2-UB_1042_3_1.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_1042_---1.jpg
2_IM-K2-UB_4020_0_1.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-U

...

/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1805572909_5511.jpg
4_IM-K2-UB_2060441482_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_2060441482_-511.jpg
4_IM-K2-UB_2227096240_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_2227096240_-511.jpg
4_IM-K2-UB_1694475249_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1694475249_5511.jpg
4_IM-K2-UB_3249961891_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_3249961891_5511.jpg
4_IM-K2-UB_1611464307_3_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1611464307_3215.jpg
2_IM-K2-UB_11286_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_11286_5522.jpg
4_IM-K2-UB_2229063408_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_2229063408_5515.jpg
4_IM-K2-UB_426285994_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_426285994_5500.jpg
4_IM-K2-UB_1591087623_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1591087623_5511.jpg
4_IM-K2-UB_69401861_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_69401861_5511.jpg
4_IM-K2-UB_3748_2_5.jpg
/mnt/data/

/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1437224381_5511.jpg
4_IM-K2-UB_1848093474_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1848093474_5511.jpg
4_IM-K2-UB_897967785_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_897967785_5511.jpg
4_IM-K2-UB_567935008_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_567935008_5515.jpg
2_IM-K2-UB_554087616_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_554087616_5521.jpg
4_IM-K2-UB_3860761402_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_3860761402_5511.jpg
4_IM-K2-UB_3521705215_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_3521705215_5511.jpg
4_IM-K2-UB_4109428802_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_4109428802_-511.jpg
4_IM-K2-UB_1036396669_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1036396669_5511.jpg
4_IM-K2-UB_921924757_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_921924757_5511.jpg
4_IM-K2-UB_1456209985_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1456209985_5512.jpg
4_IM-K2-UB_338988746_2_5.j

/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1884614208_5511.jpg
4_IM-K2-UB_1393899766_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1393899766_5511.jpg
4_IM-K2-UB_440455665_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_440455665_5511.jpg
4_IM-K2-UB_1685248844_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1685248844_5511.jpg
4_IM-K2-UB_2022978785_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_2022978785_5511.jpg
4_IM-K2-UB_1685015653_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1685015653_5515.jpg
4_IM-K2-UB_4124312687_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_4124312687_5511.jpg
4_IM-K2-UB_1747691658_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1747691658_5510.jpg
4_IM-K2-UB_18339_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_18339_55-0.jpg
4_IM-K2-UB_16391_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_16391_5511.jpg
4_IM-K2-UB_1633178639_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1633178639_5511.jpg
4_IM-K2-UB_4031020473_2_5.jpg
/mnt/data/

4_IM-K2-UB_1285569997_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1285569997_5511.jpg
4_IM-K2-UB_954806352_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_954806352_5510.jpg
4_IM-K2-UB_3249959994_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_3249959994_-511.jpg
2_IM-K2-UB_13509_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_13509_5511.jpg
4_IM-K2-UB_2081691411_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_2081691411_5511.jpg
4_IM-K2-UB_2123405776_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_2123405776_5511.jpg
2_IM-K2-UB_4301_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_4301_5312.jpg
4_IM-K2-UB_669688495_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_669688495_5511.jpg
4_IM-K2-UB_3961978007_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_3961978007_5510.jpg
4_IM-K2-UB_253060918_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_253060918_5512.jpg
4_IM-K2-UB_2265625800_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_2265625800_5511.jpg
4_IM-K2-UB_18063

/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_3373855166_5555.jpg
2_IM-K2-UB_4964_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/2_IMG-K2-UB_4964_3522.jpg
4_IM-K2-UB_10795_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_10795_5511.jpg
4_IM-K2-UB_922102829_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_922102829_5522.jpg
4_IM-K2-UB_3683458409_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_3683458409_5511.jpg
4_IM-K2-UB_673594189_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_673594189_5511.jpg
4_IM-K2-UB_4130265837_3_2.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_4130265837_5512.jpg
4_IM-K2-UB_1808538781_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_1808538781_5511.jpg
4_IM-K2-UB_326192959_2_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_326192959_5511.jpg
4_IM-K2-UB_2022429248_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_2022429248_5522.jpg
4_IM-K2-UB_698620177_0_5.jpg
/mnt/data/data/ub-im/Imgtrain/4_IMG-K2-UB_698620177_551-.jpg
2_IM-K2-UB_3720561265_0_5.jpg
/mnt/data/data/ub-

In [7]:
SPLIT_SIZE = 0.25

test_path = join(img_path, 'test/')
train_path = join(img_path, 'train/')
val_path = join(img_path, 'val/')
list_file = os.listdir(img_path)
split_size = len(list_file) - int(len(list_file) * SPLIT_SIZE)
print(split_size)
train = list_file[:split_size]
test = list_file[split_size:]
for file in train:
    if os.path.isdir(img_path + file):
        continue
    print(file)
    labels = os.path.splitext(file)[0].split('_')
    if '-' in labels[len(labels) - 1]:
        os.remove(img_path + file)
        continue
    shutil.move(img_path + file, train_path + file)
            
for file in test:
    if os.path.isdir(img_path + file):
        continue
    print(file)
    labels = os.path.splitext(file)[0].split('_')
    if '-' in labels[len(labels) - 1]:
        os.remove(img_path + file)
        continue
    shutil.move(img_path + file, test_path + file)
            
list_file = os.listdir(train_path)
split_size = len(list_file) - int(len(list_file) * SPLIT_SIZE)
val = list_file[split_size:]
for file in val:
    print(file)
    shutil.copy2(train_path + file, val_path + file)
    
list_file = os.listdir(test_path)
split_size = len(list_file) - int(len(list_file) * SPLIT_SIZE)
val = list_file[split_size:]
for file in val:
    print(file)
    shutil.copy2(test_path + file, val_path + file)
        

1785
2_IMG-K2-UB_7757_5522.jpg
2_IMG-K2-UB_11792_3322.jpg
2_IMG-K2-UB_9475_3313.jpg
2_IMG-K2-UB_6907_3322.jpg
2_IMG-K2-UB_90_3322.jpg
2_IMG-K2-UB_12403_3322.jpg
2_IMG-K2-UB_12975_3322.jpg
4_IMG-K2-UB_2163406609_5511.jpg
4_IMG-K2-UB_1094519509_5511.jpg
2_IMG-K2-UB_11343_3322.jpg
4_IMG-K2-UB_3757083901_5511.jpg
2_IMG-K2-UB_2860_3322.jpg
4_IMG-K2-UB_1806392936_5511.jpg
2_IMG-K2-UB_9347_3522.jpg
4_IMG-K2-UB_3567414364_5511.jpg
2_IMG-K2-UB_11592_3322.jpg
2_IMG-K2-UB_5098_3322.jpg
2_IMG-K2-UB_3075_3522.jpg
2_IMG-K2-UB_12531_3322.jpg
2_IMG-K2-UB_14376_3322.jpg
2_IMG-K2-UB_14484_5521.jpg
4_IMG-K2-UB_3748_5511.jpg
2_IMG-K2-UB_7097_3322.jpg
2_IMG-K2-UB_14749_3322.jpg
4_IMG-K2-UB_4001465720_5511.jpg
2_IMG-K2-UB_19780_2222.jpg
4_IMG-K2-UB_1129989785_5511.jpg
4_IMG-K2-UB_363969826_5511.jpg
2_IMG-K2-UB_2044_3522.jpg
4_IMG-K2-UB_2441426534_5511.jpg
2_IMG-K2-UB_6543_3322.jpg
2_IMG-K2-UB_8685_3322.jpg
2_IMG-K2-UB_10082_5513.jpg
4_IMG-K2-UB_2595954723_5555.jpg
4_IMG-K2-UB_4039618751_5511.jpg
2_IMG-K2-UB

4_IMG-K2-UB_1560085210_5511.jpg
4_IMG-K2-UB_195600328_5511.jpg
2_IMG-K2-UB_2935271427_2222.jpg
2_IMG-K2-UB_12516_2222.jpg
4_IMG-K2-UB_4137473194_5511.jpg
2_IMG-K2-UB_7455_3322.jpg
2_IMG-K2-UB_3952653050_3322.jpg
4_IMG-K2-UB_3279132432_5510.jpg
2_IMG-K2-UB_8042_3312.jpg
4_IMG-K2-UB_3249961891_5511.jpg
2_IMG-K2-UB_347_3411.jpg
2_IMG-K2-UB_8286_3522.jpg
2_IMG-K2-UB_318465122_3522.jpg
2_IMG-K2-UB_1302478452_3312.jpg
4_IMG-K2-UB_964535828_5555.jpg
2_IMG-K2-UB_13213_5311.jpg
2_IMG-K2-UB_17069_3322.jpg
4_IMG-K2-UB_3680215011_5512.jpg
2_IMG-K2-UB_12782_3322.jpg
2_IMG-K2-UB_16616_3322.jpg
4_IMG-K2-UB_2253473443_5511.jpg
4_IMG-K2-UB_4033081142_5511.jpg
4_IMG-K2-UB_3857234900_5511.jpg
4_IMG-K2-UB_2617676552_5511.jpg
4_IMG-K2-UB_3274654304_5500.jpg
4_IMG-K2-UB_3869113212_5511.jpg
2_IMG-K2-UB_2924_3322.jpg
4_IMG-K2-UB_1696050126_5511.jpg
2_IMG-K2-UB_11875_3531.jpg
2_IMG-K2-UB_2547000901_4422.jpg
4_IMG-K2-UB_3854490920_5522.jpg
2_IMG-K2-UB_3286_4411.jpg
2_IMG-K2-UB_16321_3322.jpg
4_IMG-K2-UB_3904359

2_IMG-K2-UB_17666_2222.jpg
4_IMG-K2-UB_1064653026_5325.jpg
2_IMG-K2-UB_16526_3333.jpg
2_IMG-K2-UB_1115_3322.jpg
4_IMG-K2-UB_1789186330_5555.jpg
2_IMG-K2-UB_17662_2422.jpg
2_IMG-K2-UB_2185_3322.jpg
4_IMG-K2-UB_3760548315_5511.jpg
4_IMG-K2-UB_1844974786_5555.jpg
4_IMG-K2-UB_2731343047_5555.jpg
2_IMG-K2-UB_4121195294_2222.jpg
4_IMG-K2-UB_675314665_5515.jpg
4_IMG-K2-UB_338534809_5511.jpg
2_IMG-K2-UB_7229_5522.jpg
2_IMG-K2-UB_7461_2222.jpg
4_IMG-K2-UB_1295399279_5511.jpg
4_IMG-K2-UB_4281602518_5511.jpg
2_IMG-K2-UB_3965_3322.jpg
4_IMG-K2-UB_3380_5511.jpg
2_IMG-K2-UB_411_3322.jpg
4_IMG-K2-UB_1158125950_5511.jpg
4_IMG-K2-UB_2693281042_5511.jpg
4_IMG-K2-UB_2397632283_5510.jpg
4_IMG-K2-UB_669795402_5515.jpg
2_IMG-K2-UB_4402_3322.jpg
4_IMG-K2-UB_2772602394_5525.jpg
4_IMG-K2-UB_829908275_5511.jpg
4_IMG-K2-UB_8490_5511.jpg
4_IMG-K2-UB_3579728661_5511.jpg
4_IMG-K2-UB_3343251777_5522.jpg
2_IMG-K2-UB_3237_5522.jpg
2_IMG-K2-UB_10715_3322.jpg
2_IMG-K2-UB_16332_3522.jpg
4_IMG-K2-UB_553001737_5511.jpg
4_I